In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def f_hook(m, i, o):
    print(m)
    print(type(m))
    print("forward_input")
    print(type(i))
    for a in i:
        print(a)
    print("forward_output")
    print(type(o))
    print(o)
    print()

def b_hook(m, i, o):
    print(m)
    print(type(m))
    print("grad_input")
    print(type(i))
    for a in i:
        print(a)
    print("grad_output")
    print(type(o))
    for a in o:
        print(a)
    print()

def tensor_hook(x):
    print(type(x))
    print(x)
    print()

## Linear

(grad_bias, grad_in, grad_weight)

In [2]:
x = torch.rand(1, 5).requires_grad_(True)
linear_layer = nn.Linear(5, 3)
relu = nn.ReLU()
linear_layer2 = nn.Linear(3, 2)

linear_layer.register_forward_hook(f_hook)
relu.register_forward_hook(f_hook)
linear_layer2.register_forward_hook(f_hook)
linear_layer.register_backward_hook(b_hook)
relu.register_backward_hook(b_hook)
linear_layer2.register_backward_hook(b_hook)
o1 = linear_layer(x)
o2 = relu(o1)
o3 = linear_layer2(o2)
o4 = 9 - o3.mean()

Linear(in_features=5, out_features=3, bias=True)
<class 'torch.nn.modules.linear.Linear'>
forward_input
<class 'tuple'>
tensor([[0.6774, 0.8721, 0.6542, 0.7704, 0.9726]], requires_grad=True)
forward_output
<class 'torch.Tensor'>
tensor([[-0.1610,  0.8677, -0.1616]], grad_fn=<AddmmBackward>)

ReLU()
<class 'torch.nn.modules.activation.ReLU'>
forward_input
<class 'tuple'>
tensor([[-0.1610,  0.8677, -0.1616]], grad_fn=<AddmmBackward>)
forward_output
<class 'torch.Tensor'>
tensor([[0.0000, 0.8677, 0.0000]], grad_fn=<ReluBackward0>)

Linear(in_features=3, out_features=2, bias=True)
<class 'torch.nn.modules.linear.Linear'>
forward_input
<class 'tuple'>
tensor([[0.0000, 0.8677, 0.0000]], grad_fn=<ReluBackward0>)
forward_output
<class 'torch.Tensor'>
tensor([[-0.6003,  0.1332]], grad_fn=<AddmmBackward>)



In [3]:
o4.backward()

Linear(in_features=3, out_features=2, bias=True)
<class 'torch.nn.modules.linear.Linear'>
grad_input
<class 'tuple'>
tensor([-0.5000, -0.5000])
tensor([[0.1422, 0.2067, 0.1685]])
tensor([[-0.0000, -0.0000],
        [-0.4339, -0.4339],
        [-0.0000, -0.0000]])
grad_output
<class 'tuple'>
tensor([[-0.5000, -0.5000]])

ReLU()
<class 'torch.nn.modules.activation.ReLU'>
grad_input
<class 'tuple'>
tensor([[0.0000, 0.2067, 0.0000]])
grad_output
<class 'tuple'>
tensor([[0.1422, 0.2067, 0.1685]])

Linear(in_features=5, out_features=3, bias=True)
<class 'torch.nn.modules.linear.Linear'>
grad_input
<class 'tuple'>
tensor([0.0000, 0.2067, 0.0000])
tensor([[ 0.0744, -0.0104, -0.0731,  0.0776,  0.0899]])
tensor([[0.0000, 0.1400, 0.0000],
        [0.0000, 0.1802, 0.0000],
        [0.0000, 0.1352, 0.0000],
        [0.0000, 0.1592, 0.0000],
        [0.0000, 0.2010, 0.0000]])
grad_output
<class 'tuple'>
tensor([[0.0000, 0.2067, 0.0000]])



## Conv2d

In [4]:
x = torch.randn(1, 1, 5, 5)
conv_layer = nn.Conv2d(1, 1, 2, 1)
relu = nn.ReLU()
conv_layer2 = nn.Conv2d(1, 1, 2, 1)

conv_layer.register_forward_hook(f_hook)
relu.register_forward_hook(f_hook)
conv_layer2.register_forward_hook(f_hook)
conv_layer.register_backward_hook(b_hook)
relu.register_backward_hook(b_hook)
conv_layer2.register_backward_hook(b_hook)
o1 = conv_layer(x)
o2 = relu(o1)
o3 = conv_layer2(o2)
o4 = 1 - o3.mean()

Conv2d(1, 1, kernel_size=(2, 2), stride=(1, 1))
<class 'torch.nn.modules.conv.Conv2d'>
forward_input
<class 'tuple'>
tensor([[[[-2.2428, -0.6507, -0.9266, -0.2799, -0.2403],
          [ 0.6220, -1.7246,  0.7825, -0.2940, -2.0784],
          [-0.3635,  0.7736,  0.1153, -0.6302, -1.0533],
          [-1.7638,  0.2802, -0.1999,  1.4780,  2.0490],
          [-0.5662,  1.8826, -0.3097, -1.5586, -0.4505]]]])
forward_output
<class 'torch.Tensor'>
tensor([[[[-0.8464, -0.9336, -0.7355,  0.0259],
          [-0.7275, -0.9102, -0.0453, -0.5891],
          [-0.3774, -0.0958, -0.8881, -1.4553],
          [-1.4319, -0.3699,  0.2377,  0.6649]]]],
       grad_fn=<MkldnnConvolutionBackward>)

ReLU()
<class 'torch.nn.modules.activation.ReLU'>
forward_input
<class 'tuple'>
tensor([[[[-0.8464, -0.9336, -0.7355,  0.0259],
          [-0.7275, -0.9102, -0.0453, -0.5891],
          [-0.3774, -0.0958, -0.8881, -1.4553],
          [-1.4319, -0.3699,  0.2377,  0.6649]]]],
       grad_fn=<MkldnnConvolutionBackward>

In [5]:
o4.backward()

Conv2d(1, 1, kernel_size=(2, 2), stride=(1, 1))
<class 'torch.nn.modules.conv.Conv2d'>
grad_input
<class 'tuple'>
tensor([[[[ 0.0393,  0.0407,  0.0407,  0.0015],
          [-0.0095, -0.0455, -0.0455, -0.0360],
          [-0.0095, -0.0455, -0.0455, -0.0360],
          [-0.0488, -0.0863, -0.0863, -0.0375]]]])
tensor([[[[ 0.0000, -0.0029],
          [-0.0264, -0.1003]]]])
tensor([-1.])
grad_output
<class 'tuple'>
tensor([[[[-0.1111, -0.1111, -0.1111],
          [-0.1111, -0.1111, -0.1111],
          [-0.1111, -0.1111, -0.1111]]]])

ReLU()
<class 'torch.nn.modules.activation.ReLU'>
grad_input
<class 'tuple'>
tensor([[[[ 0.0000,  0.0000,  0.0000,  0.0015],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000, -0.0863, -0.0375]]]])
grad_output
<class 'tuple'>
tensor([[[[ 0.0393,  0.0407,  0.0407,  0.0015],
          [-0.0095, -0.0455, -0.0455, -0.0360],
          [-0.0095, -0.0455, -0.0455, -0.0360],
          [-0.0488, -

## MaxPool2d

In [9]:
x = torch.randn(1, 1, 8, 8).requires_grad_(True)
pool_layer = nn.MaxPool2d(2)
relu = nn.ReLU()
pool_layer2 = nn.MaxPool2d(2)

pool_layer.register_forward_hook(f_hook)
relu.register_forward_hook(f_hook)
pool_layer2.register_forward_hook(f_hook)
pool_layer.register_backward_hook(b_hook)
relu.register_backward_hook(b_hook)
pool_layer2.register_backward_hook(b_hook)
o1 = pool_layer(x)
o2 = relu(o1)
o3 = pool_layer2(o2)
o4 = 1 - o3.mean()

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
<class 'torch.nn.modules.pooling.MaxPool2d'>
forward_input
<class 'tuple'>
tensor([[[[ 0.2773,  0.3788, -0.8474,  1.6525, -1.4269, -0.1751, -0.7948,
            0.2649],
          [-0.5877,  1.7464,  0.4918, -1.7818,  0.0117, -1.9523, -1.2985,
           -0.9928],
          [-0.0988, -0.1853,  0.1200,  1.1873,  0.8811, -2.5908, -0.8597,
            2.3402],
          [ 0.1572, -0.2378,  0.3196,  0.0429, -0.1465,  0.7755, -0.3924,
           -0.3190],
          [ 0.0209, -0.5632, -1.1840,  0.2455,  1.0940, -0.0181, -0.2725,
            2.7206],
          [ 0.0907, -0.2583,  1.3619,  0.2521,  0.7138,  0.7075, -0.9619,
            0.6584],
          [-0.4279, -1.5434, -0.6213, -1.6734,  0.4150,  0.1512, -0.0308,
           -1.2621],
          [-1.4041,  1.6359,  0.2737, -1.0433,  0.1063,  1.4708,  1.3754,
            0.1227]]]], requires_grad=True)
forward_output
<class 'torch.Tensor'>
tensor([[[[1.7464, 1.6525, 0.

In [10]:
o4.backward()

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
<class 'torch.nn.modules.pooling.MaxPool2d'>
grad_input
<class 'tuple'>
tensor([[[[-0.2500,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000, -0.2500],
          [ 0.0000,  0.0000,  0.0000, -0.2500],
          [-0.2500,  0.0000,  0.0000,  0.0000]]]])
grad_output
<class 'tuple'>
tensor([[[[-0.2500, -0.2500],
          [-0.2500, -0.2500]]]])

ReLU()
<class 'torch.nn.modules.activation.ReLU'>
grad_input
<class 'tuple'>
tensor([[[[-0.2500,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000, -0.2500],
          [ 0.0000,  0.0000,  0.0000, -0.2500],
          [-0.2500,  0.0000,  0.0000,  0.0000]]]])
grad_output
<class 'tuple'>
tensor([[[[-0.2500,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000, -0.2500],
          [ 0.0000,  0.0000,  0.0000, -0.2500],
          [-0.2500,  0.0000,  0.0000,  0.0000]]]])

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=Fa

## RNN

In [4]:
torch.manual_seed(5)
x = torch.randn(1, 2, 5).requires_grad_(True)
rnn_layer = nn.RNN(5, 3, 1, batch_first=True)
softmax = nn.Softmax(dim=1)
 
rnn_layer.register_forward_hook(f_hook)
# softmax.register_forward_hook(f_hook)
rnn_layer.register_backward_hook(b_hook)
# softmax.register_backward_hook(b_hook)
rnn_layer.weight_hh_l0.register_hook(tensor_hook)
rnn_layer.weight_ih_l0.register_hook(tensor_hook)
o1, h1 = rnn_layer(x)
o2 = softmax(h1.squeeze(1))
grad = torch.FloatTensor([[1, 0, 0]])

RNN(5, 3, batch_first=True)
<class 'torch.nn.modules.rnn.RNN'>
forward_input
<class 'tuple'>
tensor([[[-0.4868, -0.6038, -0.5581,  0.6675, -0.1974],
         [ 1.9428, -1.4017, -0.7626,  0.6312, -0.8991]]], requires_grad=True)
forward_output
<class 'tuple'>
tensor([[[ 0.7737,  0.6712,  0.7200],
         [ 0.9293,  0.0850, -0.2211]]], grad_fn=<TransposeBackward1>)
tensor([[[ 0.9293,  0.0850, -0.2211]]], grad_fn=<StackBackward>)



In [5]:
l = F.cross_entropy(o1.view(-1, 3), torch.LongTensor([1, 0]))
l.backward()

RNN(5, 3, batch_first=True)
<class 'torch.nn.modules.rnn.RNN'>
grad_input
<class 'tuple'>
tensor([[[ 0.1754, -0.3417,  0.1662]],

        [[-0.2137,  0.1231,  0.0906]]])
grad_output
<class 'tuple'>
tensor([[[ 0.1754, -0.3417,  0.1662],
         [-0.2137,  0.1231,  0.0906]]])

<class 'torch.Tensor'>
tensor([[-0.0225, -0.0196, -0.0210],
        [ 0.0945,  0.0820,  0.0880],
        [ 0.0667,  0.0579,  0.0621]])
<class 'torch.Tensor'>
tensor([[-0.0849,  0.0058, -0.0101,  0.0203,  0.0148],
        [ 0.3266, -0.0606,  0.0091, -0.0452, -0.0737],
        [ 0.1169, -0.1835, -0.1237,  0.1238, -0.0980]])
